#################################################################################################################################################################################################################################################################################################################################################
# Collaboration and Competition
---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
import torch
import numpy as np
from unityagents import UnityEnvironment
import os

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis.app", worker_id=303)

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [4]:
for i in range(1, 5):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

Score (max over agents) from episode 1: 0.0
Score (max over agents) from episode 2: 0.0
Score (max over agents) from episode 3: 0.0
Score (max over agents) from episode 4: 0.0


When finished, you can close the environment.

In [5]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [6]:
from ma_ddpg import MADDPG

model_dir = 'saved_models/'
model_name = 'multi-agents_' + str(brain_name) + '_' + str(num_agents) + '_agents.pt'

os.makedirs(model_dir, exist_ok=True)

episode_max = 5000
rollout_len = 1000

ma_ddpg = MADDPG(state_size, action_size, num_agents, seed=12345)

In [7]:
def saveTrainedModel(agent, path):
    state_dicts = {}
    for i in range(num_agents):
        state_dicts['model_agent' + str(i)] = {'actor_local': ma_ddpg.agents_list[i].actor_local.state_dict(), 
                                               'critic_local': ma_ddpg.agents_list[i].critic_local.state_dict(), 
                                               'actor_target': ma_ddpg.agents_list[i].actor_target.state_dict(), 
                                               'critic_target': ma_ddpg.agents_list[i].critic_target.state_dict()}
    torch.save(state_dicts, path)

In [ ]:
agents_scores_history = [[] for _ in range(num_agents)]
max_episodic_scores_history = []
average_scores_history = []
t_history = []
e = 0
isSolved = False
solved_episode = -1

while e < episode_max:
    
    env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
    ma_ddpg.noise_reset()                                  # reset noise of the agent    
    states = env_info.vector_observations                  # get the current state np array :num_agents x state_size
    running_score = np.zeros(num_agents)                   # initialize the score (for each agent)
    
    t = 0
    
    while True:
        actions_list = ma_ddpg.acts(states)                # select an action (for each agent)
        actor_tensor = torch.stack(actions_list).cpu().detach()
        actions = np.clip(actor_tensor.numpy(), -1, 1)     # all actions between -1 and 1 #num_agents x action_size
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (# num_agents x state_size)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        
        ma_ddpg.step((states, actor_tensor, rewards, dones, next_states))
        
        running_score += rewards                           # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones) or t >= rollout_len:   
            t_history.append(t)
            break
            
        t += 1

    for i in range(num_agents):
        agents_scores_history[i].append(running_score[i])
        
    max_episodic_scores_history.append(np.max(running_score))
    
    avg_100_score = np.mean(max_episodic_scores_history[-100:])
    average_scores_history.append(avg_100_score)

    if ma_ddpg.is_training:
        if (e+1)%20==0:
            print("e: {}  score(s): {:.3f} {:.3f}  Avg score: {:.3f}  "
                  "Actor(g): {:.2e}  Critic(l): {:.2e}  n: {:.2f}  "
                  "td: {:.3f}".format(e+1, np.mean(agents_scores_history[0][-100:]), 
                                     np.mean(agents_scores_history[1][-100:]), 
                                     avg_100_score,
                                     np.mean(ma_ddpg.ag_history), 
                                     np.mean(ma_ddpg.cl_history),
                                     np.mean(ma_ddpg.noise_history), 
                                     np.mean(ma_ddpg.td_history)))
            
        if not isSolved and avg_100_score >= 0.5:
            print('\033[32m env solved at episode {}! Average Score: '.format(e-100, avg_100_score))
            isSolved = True
        if isSolved and e >= solved_episode+100:
            break #run for 100 more episodes after solving   
            
        e+=1
    else:
        print('\rFetching experiences... {} '.format(ma_ddpg.t_step), end="")

Fetching experiences... 990 Prefetch experience completed. Training starts! 
Number of Agents:  2
Device:  cpu
BATCH_SIZE:  512
LR_ACTOR:  0.0001
LR_CRITIC:  0.0001
REWARD_NORM:  False
REWARD_SCALE:  1.0
LEARN_EVERY:  20
LEARN_LOOP:  10
UNITS_ACTOR:  (512, 256)
UNITS_CRITIC:  (512, 256)
NOISE_WGT_INIT:  1.0
NOISE_WGT_DECAY:  0.9995
NOISE_DECAY_EVERY:  5
NOISE_WGT_MIN:  0.15
TARGET_NOISE:  False
USE_PER:  True
CRITIC_ACT_FORM:  1
e: 20  score(s): -0.002 -0.002  Avg score: 0.006  Actor(g): 6.68e-03  Critic(l): 2.58e-05  n: 0.30  td: 0.011
e: 40  score(s): -0.001 -0.001  Avg score: 0.008  Actor(g): 6.68e-03  Critic(l): 2.15e-05  n: 0.24  td: 0.010
e: 60  score(s): -0.000 -0.002  Avg score: 0.007  Actor(g): 6.04e-03  Critic(l): 1.85e-05  n: 0.27  td: 0.009
e: 80  score(s): 0.001 0.001  Avg score: 0.009  Actor(g): 5.77e-03  Critic(l): 1.65e-05  n: 0.26  td: 0.009
e: 100  score(s): 0.002 0.001  Avg score: 0.011  Actor(g): 5.42e-03  Critic(l): 1.55e-05  n: 0.22  td: 0.009
e: 120  score(s): 0.

In [ ]:
#target no noise but min is tiny
saveTrainedModel(ma_ddpg, model_dir + model_name)

In [ ]:
# plot the scores
import matplotlib.pyplot as plt
%matplotlib inline

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(max_episodic_scores_history)), max_episodic_scores_history)
plt.plot(np.arange(len(average_scores_history)), average_scores_history) # plotting t, c separately 
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
len(average_scores_history)